# Reference

    1. Automatic Chemical Design Using a Data-Driven Continuous Representation of Molecules, 2015 ( https://pubs.acs.org/doi/full/10.1021/acscentsci.7b00572 )
        ✔ SMILES VAE는 분자를 연속적이고 차별화 가능한 latent space에 매핑하며, 특정 속성(logP, QED, SAS, 생체 활성 등)에 최적화할 수 있음.

    2. Self-Referencing Embedded Strings (SELFIES): A 100% robust molecular string representation, 2020 ( https://iopscience.iop.org/article/10.1088/2632-2153/aba947 )
        ✔ language model로 분자를 생성할 때, invalid한 분자를 생성할 가능성이 있는 SMILES 표기와 달리, SELIFES 표기를 사용하여 100% valid한 분자 생성을 하도록 함.

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

Tue Aug 22 04:34:59 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install wandb
!pip install rdkit
!pip install selfies

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.6/215.6 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.1 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=978e146963093f2fa3033d4ab656a035ddfb499b28d4eb68d869e3c428992f23
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 47.8 MB/s eta 0:00:00


In [ ]:
import argparse
import sys
import yaml
import json
import os
import joblib
import networkx as nx
from tqdm import tqdm
from datetime import datetime
import time
import random
import copy
from collections import defaultdict
import pickle as pkl
import re
import spacy
import codecs
import collections.abc
collections.Mapping = collections.abc.Mapping

from prettytable import PrettyTable
import selfies
import wandb

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import rdkit
import rdkit.Chem as Chem
from rdkit.Chem import Draw
import rdkit.Chem.AllChem as AllChem
from PIL import Image
from IPython.core.display import display

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn.utils import clip_grad_norm_

import torchvision
import torchvision.transforms as transforms

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

os.environ['TORCH'] = torch.__version__
print(f'Torch ver. : {torch.__version__}')

Torch ver. : 2.0.1+cu118


In [ ]:
ENTITY = 'rossi22'
TRAIN_PROJECT = 'TRANS_VAE_SELFIES_TRAIN'
SWEEP_PROJECT = 'TRANS_VAE_SELFIES_SWEEP'

DEFAULT_PARAMS = {
    'device': torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    'batch_size': 128,
    'learning_rate': 0.0002,
    'epochs': 500,
    'period':5,

    'embed_dim': 32,
    'latent_dim': 128,
    'hidden_dim': 64,
    'enc_layers': 4,
    'dec_layers': 4,
    'enc_heads': 8,
    'dec_heads': 8,
    'enc_pos_dim': 256,
    'dec_pos_dim': 256,
    'enc_dropout': 0.1,
    'dec_dropout': 0.1,
    'pad_idx': 0,
    'clip': 0.01,

    'use_scheduler': True,
    'schd_factor': 0.5,
    'schd_patience': 10,
    'schd_min_lr': 5e-6,
}

In [ ]:
def load_smiles(file_name, num_mol):
    file_pkl = pkl.load(open(file_name, 'rb'))
    num_mol = min(len(file_pkl), num_mol)
    smi_list = []
    selfies_list = []
    for i in tqdm(range(num_mol), desc='Reading Data'):
        smi = file_pkl[i].strip()
        smi_list.append(smi)
        selfies_list.append(selfies.encoder(smi))

    return smi_list, selfies_list

In [ ]:
_, selfies_list = load_smiles('/content/drive/MyDrive/Colab Notebooks/Python/Portfolio/dataset/ZINC_smi.pkl', int(3e5))
_, selfies_list2 = load_smiles('/content/drive/MyDrive/Colab Notebooks/Python/Portfolio/dataset/ZINC_smi2.pkl', int(3e5))
selfies_list = list(set(selfies_list+selfies_list2))
print(len(selfies_list))

Reading Data: 100%|██████████| 251637/251637 [01:26<00:00, 2899.70it/s]


350867


In [ ]:
def dict_tokens(data) :
    tokens = {'<pad>': 0, '<sos>': 1, '<eos>': 2, }
    max_len = 0
    for word in data :
        max_len = max(max_len, selfies.len_selfies(word))

    selfies_token_list = selfies.get_alphabet_from_selfies(data)
    selfies_tokens = {key:i+3 for i, key in enumerate(selfies_token_list)}
    tokens.update(selfies_tokens)
    selfies_r_tokens = {val:key for (key, val) in tokens.items()}

    return max_len+2, tokens, selfies_r_tokens

DEFAULT_PARAMS['selfies_max_len'], DEFAULT_PARAMS['selfies_tokens'], DEFAULT_PARAMS['selfies_reverse_tokens'] = dict_tokens(selfies_list)

In [ ]:
def embedding_data(data, max_len=DEFAULT_PARAMS['selfies_max_len'], dict_tokens=DEFAULT_PARAMS['selfies_tokens']) :
    emb_data = torch.zeros((len(data), max_len))

    for idx1, word in tqdm(enumerate(data)) :
        emb_data[idx1][0] = dict_tokens['<sos>']
        idx2 = 1
        for tok in selfies.split_selfies(word) :
            if tok != '.' :
                emb_data[idx1][idx2] = dict_tokens[tok]
                idx2 +=1
        emb_data[idx1][idx2] = dict_tokens['<eos>']

    emb_data = emb_data.type(torch.LongTensor)
    return emb_data

embedded_selfies = embedding_data(selfies_list)

print(selfies_list[0])
print(embedded_selfies[0])

350867it [01:25, 4095.33it/s]


[C][C][=C][C][=C][Branch1][#C][C][=Branch1][C][=O][O][C][C][C][N][C][Branch1][C][N][=O][C][=C][Ring1][S][C]
tensor([ 1, 15, 15, 37, 15, 37,  3, 35, 15, 16, 15, 34, 29, 15, 15, 15, 41, 15,
         3, 15, 41, 34, 15, 37, 38, 36, 15,  2,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0])


In [ ]:
train_size = int(len(selfies_list) * 9.5 // 10)
val_size = (len(selfies_list)-train_size)//2

def set_seed(seed_value):

    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

    np.random.seed(seed_value)
    random.seed(seed_value)

set_seed(52)
random.shuffle(embedded_selfies)

TRAIN_SET, VAL_SET, TEST_SET = embedded_selfies[:train_size], embedded_selfies[train_size:train_size+val_size], embedded_selfies[train_size+val_size:]
print(len(TRAIN_SET), len(VAL_SET), len(TEST_SET))

333323 8772 8772


In [ ]:
class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hidden_dim, n_heads, dropout_ratio, device):
        super().__init__()

        assert hidden_dim % n_heads == 0

        self.hidden_dim = hidden_dim # 임베딩 차원
        self.n_heads = n_heads # 헤드(head)의 개수: 서로 다른 어텐션(attention) 컨셉의 수
        self.head_dim = hidden_dim // n_heads # 각 헤드(head)에서의 임베딩 차원

        self.fc_q = nn.Linear(hidden_dim, hidden_dim) # Query 값에 적용될 FC 레이어
        self.fc_k = nn.Linear(hidden_dim, hidden_dim) # Key 값에 적용될 FC 레이어
        self.fc_v = nn.Linear(hidden_dim, hidden_dim) # Value 값에 적용될 FC 레이어

        self.fc_o = nn.Linear(hidden_dim, hidden_dim)

        self.dropout = nn.Dropout(dropout_ratio)

        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)

    def forward(self, query, key, value, mask = None):

        batch_size = query.shape[0]

        # query: [batch_size, query_len, hidden_dim]
        # key: [batch_size, key_len, hidden_dim]
        # value: [batch_size, value_len, hidden_dim]

        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)

        # Q: [batch_size, query_len, hidden_dim]
        # K: [batch_size, key_len, hidden_dim]
        # V: [batch_size, value_len, hidden_dim]

        # hidden_dim → n_heads X head_dim 형태로 변형
        # n_heads(h)개의 서로 다른 어텐션(attention) 컨셉을 학습하도록 유도
        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)

        # Q: [batch_size, n_heads, query_len, head_dim]
        # K: [batch_size, n_heads, key_len, head_dim]
        # V: [batch_size, n_heads, value_len, head_dim]

        # [1], [2] Attention Energy 계산, scaling
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale

        # energy: [batch_size, n_heads, query_len, key_len]

        # [3] 마스크(mask)를 사용하는 경우
        if mask is not None:
            # 마스크(mask) 값이 0인 부분을 -1e10으로 채우기
            energy = energy.masked_fill(mask==0, -1e10)

        # [4] 어텐션(attention) 스코어 계산: 각 단어에 대한 확률 값
        attention = torch.softmax(energy, dim=-1)
        # attention: [batch_size, n_heads, query_len, key_len]

        # [5] attention과 Value의 Scaled Dot-Product Attention을 계산
        x = torch.matmul(self.dropout(attention), V)
        # x: [batch_size, n_heads, query_len, head_dim]
        x = x.permute(0, 2, 1, 3).contiguous()
        # x: [batch_size, query_len, n_heads, head_dim]
        x = x.view(batch_size, -1, self.hidden_dim)
        # x: [batch_size, query_len, hidden_dim]
        x = self.fc_o(x)
        # x: [batch_size, query_len, hidden_dim]
        return x, attention

class PositionwiseFeedforwardLayer(nn.Module):
    def __init__(self, hidden_dim, pos_dim, dropout_ratio):
        super().__init__()

        self.fc_1 = nn.Linear(hidden_dim, pos_dim)
        self.fc_2 = nn.Linear(pos_dim, hidden_dim)
        self.dropout = nn.Dropout(dropout_ratio)

    def forward(self, x):
        # x: [batch_size, seq_len, hidden_dim]
        x = self.dropout(torch.relu(self.fc_1(x)))
        # x: [batch_size, seq_len, pos_dim]
        x = self.fc_2(x)
        # x: [batch_size, seq_len, hidden_dim]
        return x

class GatedSkipConnection(nn.Module):

    def __init__(self, in_dim, out_dim):
        super(GatedSkipConnection, self).__init__()

        self.in_dim = in_dim
        self.out_dim = out_dim

        self.linear = nn.Linear(in_dim, out_dim, bias=False)
        self.linear_coef_in = nn.Linear(out_dim, out_dim)
        self.linear_coef_out = nn.Linear(out_dim, out_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, in_x, out_x):
        if (self.in_dim != self.out_dim):
            in_x = self.linear(in_x)
        # z : 0 ~ 1 사이의 값
        z = self.gate_coefficient(in_x, out_x)
        out = torch.mul(z, out_x) + torch.mul(1.0-z, in_x)
        return out

    def gate_coefficient(self, in_x, out_x):
        x1 = self.linear_coef_in(in_x)
        x2 = self.linear_coef_out(out_x)
        return self.sigmoid(x1+x2)

class EncoderLayer(nn.Module):
    def __init__(self, hidden_dim, n_heads, pos_dim, dropout_ratio, device):
        super().__init__()

        self.self_attn_layer_norm = nn.LayerNorm(hidden_dim)
        self.self_attention = MultiHeadAttentionLayer(hidden_dim, n_heads, dropout_ratio, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hidden_dim, pos_dim, dropout_ratio)
        self.ff_layer_norm = nn.LayerNorm(hidden_dim)
        self.dropout = nn.Dropout(dropout_ratio)
        self.gsc = GatedSkipConnection(hidden_dim, hidden_dim)

    # 하나의 임베딩이 복제되어 Query, Key, Value로 입력되는 방식
    def forward(self, src, src_mask):
        # src: [batch_size, src_len, hidden_dim]
        # src_mask: [batch_size, src_len]

        # self attention
        # 필요한 경우 마스크(mask) 행렬을 이용하여 어텐션(attention)할 단어를 조절 가능
        _src, _ = self.self_attention(src, src, src, src_mask)
        # dropout, residual connection and layer norm
        src = self.self_attn_layer_norm(self.gsc(src, _src))
        # src: [batch_size, src_len, hidden_dim]

        # position-wise feedforward
        _src = self.positionwise_feedforward(src)
        # dropout, residual and layer norm
        src = self.ff_layer_norm(self.gsc(src, _src))
        # src: [batch_size, src_len, hidden_dim]

        return src

class Encoder(nn.Module):
    def __init__(self, input_dim, embed_dim, hidden_dim, latent_dim,  n_layers, n_heads, pos_dim, dropout_ratio, max_len, device):
        super().__init__()

        self.device = device
        self.tok_embedding = nn.Embedding(input_dim, embed_dim)
        self.pos_embedding = nn.Embedding(max_len, embed_dim)
        self.initial_linear = nn.Linear(embed_dim, hidden_dim)
        self.en_layers = nn.ModuleList([EncoderLayer(hidden_dim, n_heads, pos_dim, dropout_ratio, device) for _ in range(n_layers)])
        self.dropout = nn.Dropout(dropout_ratio)
        self.scale = torch.sqrt(torch.FloatTensor([embed_dim])).to(device)

        self.relu = nn.ReLU()
        self.mean = nn.Linear(max_len * hidden_dim, latent_dim)
        self.log_var = nn.Linear(max_len * hidden_dim, latent_dim)

    def forward(self, src, src_mask):

        # src: [batch_size, src_len]
        # src_mask: [batch_size, src_len]
        batch_size = src.shape[0]
        src_len = src.shape[1]

        pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
        # pos: [batch_size, src_len]

        # 소스 문장의 임베딩과 위치 임베딩을 더한 것을 사용
        src = self.dropout((self.tok_embedding(src) * self.scale) + self.pos_embedding(pos))
        # src: [batch_size, src_len, embed_dim]
        src = self.initial_linear(src)
        # 모든 인코더 레이어를 차례대로 거치면서 순전파(forward) 수행
        for layer in self.en_layers:
            src = layer(src, src_mask)

        encoded = src.contiguous().view(src.size(0), -1)
        # encoded: [batch_size, src_len * hidden_dim]

        mean = self.relu(self.mean(encoded))
        log_var = self.relu(self.log_var(encoded))
        # mean, log_var: [batch_size, latent_dim]

        return mean, log_var

class DecoderLayer(nn.Module):
    def __init__(self, hidden_dim, n_heads, pos_dim, dropout_ratio, device):
        super().__init__()

        self.self_attn_layer_norm = nn.LayerNorm(hidden_dim)
        self.enc_attn_layer_norm = nn.LayerNorm(hidden_dim)
        self.ff_layer_norm = nn.LayerNorm(hidden_dim)
        self.self_attention = MultiHeadAttentionLayer(hidden_dim, n_heads, dropout_ratio, device)
        self.encoder_attention = MultiHeadAttentionLayer(hidden_dim, n_heads, dropout_ratio, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hidden_dim, pos_dim, dropout_ratio)
        self.dropout = nn.Dropout(dropout_ratio)
        self.gsc = GatedSkipConnection(hidden_dim, hidden_dim)

    # 인코더의 출력 값(enc_src)을 어텐션(attention)하는 구조
    def forward(self, trg, enc_src, trg_mask, src_mask):

        # trg: [batch_size, trg_len, hidden_dim]
        # enc_src: [batch_size, src_len, hidden_dim]
        # trg_mask: [batch_size, 1, src_len-1, src_len-1]
        # src_mask: [batch_size, 1, 1, src_len]

        # self attention
        # 자기 자신에 대하여 어텐션(attention)
        _trg, _ = self.self_attention(trg, trg, trg, trg_mask)

        # dropout, residual connection and layer norm
        trg = self.self_attn_layer_norm(trg + self.dropout(_trg))

        # trg: [batch_size, trg_len, hidden_dim]

        # encoder attention
        # 디코더의 쿼리(Query)를 이용해 인코더를 어텐션(attention)
        _trg, attention = self.encoder_attention(trg, enc_src, enc_src, src_mask)

        # dropout, residual connection and layer norm
        trg = self.enc_attn_layer_norm(self.gsc(trg, _trg))
        # trg = self.enc_attn_layer_norm(trg + self.dropout(_trg))
        # trg: [batch_size, trg_len, hidden_dim]

        # positionwise feedforward
        _trg = self.positionwise_feedforward(trg)

        # dropout, residual and layer norm
        trg = self.ff_layer_norm(self.gsc(trg, _trg))

        # trg = self.ff_layer_norm(trg + self.dropout(_trg))
        # trg: [batch_size, trg_len, hidden_dim]
        # attention: [batch_size, n_heads, trg_len, src_len]

        return trg, attention

class Decoder(nn.Module):
    def __init__(self, output_dim, embed_dim, hidden_dim, latent_dim, n_layers, n_heads, pos_dim, dropout_ratio, max_len, device):
        super().__init__()

        self.device = device
        self.hidden_dim = hidden_dim
        self.max_len = max_len
        self.latent_layer = nn.Linear(latent_dim, max_len*hidden_dim)
        self.relu = nn.ReLU()

        self.tok_embedding = nn.Embedding(output_dim, embed_dim)
        self.pos_embedding = nn.Embedding(max_len, embed_dim)
        self.initial_linear = nn.Linear(embed_dim, hidden_dim)
        self.layers = nn.ModuleList([DecoderLayer(hidden_dim, n_heads, pos_dim, dropout_ratio, device) for _ in range(n_layers)])
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout_ratio)
        self.scale = torch.sqrt(torch.FloatTensor([hidden_dim])).to(device)

    def forward(self, trg, z, trg_mask, src_mask):

        # z: [batch_size, latent_dim]
        # trg: [batch_size, src_len-1]
        # enc_src: [batch_size, src_len, hidden_dim]
        # trg_mask: [batch_size, src_len-1]
        # src_mask: [batch_size, src_len]

        batch_size = trg.shape[0]
        trg_len = trg.shape[1]

        z = self.relu(self.latent_layer(z))
        enc_src = z.view(batch_size, self.max_len, self.hidden_dim)

        pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
        # pos: [batch_size, trg_len]

        trg = self.dropout((self.tok_embedding(trg) * self.scale) + self.pos_embedding(pos))
        # trg: [batch_size, trg_len, embed_dim]

        trg = self.initial_linear(trg)

        for layer in self.layers:
            # 소스 마스크와 타겟 마스크 모두 사용
            trg, attention = layer(trg, enc_src, trg_mask, src_mask)
        # trg: [batch_size, trg_len, hidden_dim]
        # attention: [batch_size, n_heads, trg_len, src_len]

        output = self.fc_out(trg)
        # output: [batch_size, trg_len, output_dim]

        return output, attention

class TransformerVAE(nn.Module):
    def __init__(self, args) :
        super().__init__()

        self.encoder = Encoder(len(args['selfies_tokens']), args['embed_dim'], args['hidden_dim'], args['latent_dim'], args['enc_layers'], args['enc_heads'], args['enc_pos_dim'], args['enc_dropout'], args['selfies_max_len'], args['device'])
        self.decoder = Decoder(len(args['selfies_tokens']), args['embed_dim'], args['hidden_dim'], args['latent_dim'], args['dec_layers'], args['dec_heads'], args['dec_pos_dim'], args['dec_dropout'], args['selfies_max_len'], args['device'])
        self.pad_idx = args['pad_idx']
        self.device = device
        self.latent_dim = args['latent_dim']
        self.recon_criterion = nn.CrossEntropyLoss(ignore_index=args['pad_idx'])

    # 소스 문장의 <pad> 토큰에 대하여 마스크(mask) 값을 0(False)으로 설정
    def make_src_mask(self, src):
        # src: [batch_size, src_len]
        src_mask = (src != self.pad_idx).unsqueeze(1).unsqueeze(2)
        # src_mask: [batch_size, 1, 1, src_len]
        return src_mask

    # 타겟 문장에서 각 단어는 다음 단어가 무엇인지 알 수 없도록(이전 단어만 보도록) 만들기 위해 마스크를 사용
    def make_trg_mask(self, trg):
        # trg: [batch_size, src_len-1]

        trg_pad_mask = (trg != self.pad_idx).unsqueeze(1).unsqueeze(2)
        # trg_pad_mask: [batch_size, 1, 1, src_len-1]

        trg_len = trg.shape[1]
        trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device = self.device)).bool()
        # trg_sub_mask: [src_len-1, src_len-1]
        trg_mask = trg_pad_mask & trg_sub_mask
        # trg_mask: [batch_size, 1, src_len-1, src_len-1]
        return trg_mask

    def reparameterize(self, mean, log_var) :
        eps = torch.randn(mean.size(0), self.latent_dim).to(device)
        z = eps * torch.exp(log_var * .5) + mean
        return z

    def vae_loss(self, x, mean, log_var, z, x_hat) :
        x_hat = x_hat.contiguous().view(-1, x_hat.shape[-1])
        x = x.contiguous().view(-1)

        recon_loss = self.recon_criterion(x_hat, x)
        kl_loss = 0.5 * torch.sum(torch.square(mean) + torch.exp(log_var) - log_var -1)

        return recon_loss + kl_loss

    def forward(self, src):
        dec_src = src[:, 1:]
        trg = src[:, :-1]
        # src: [batch_size, len_src]
        # dec_src: [batch_size, len_src-1]
        # trg: [batch_size, len_src-1]

        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        # src_mask: [batch_size, 1, 1, len_src]
        # trg_mask: [batch_size, 1, len_src-1, len_src-1]

        mean, log_var = self.encoder(src, src_mask)
        z = self.reparameterize(mean, log_var)
        # z: [batch_size, latent_dim]
        # enc_src: [batch_size, len_src, hidden_dim]

        src_hat, attention = self.decoder(trg, z, trg_mask, src_mask)
        loss = self.vae_loss(dec_src, mean, log_var, z, src_hat)
        # output: [batch_size, len_src-1, output_dim]
        # attention: [batch_size, n_heads, len_src-1, len_src]
        return src_hat, attention, loss

In [ ]:
def run_train(model, train_iterator, val_iterator, optimizer, clip, device):
    epoch_train_loss = 0
    epoch_val_loss = 0

    # train
    model.train()

    for i, tokens in enumerate(train_iterator):
        tokens = tokens.to(device)
        optimizer.zero_grad()
        _, _, train_loss = model(tokens)

        train_loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()

        epoch_train_loss += train_loss.item()

    # validation
    model.eval()

    with torch.no_grad() :
        for i, tokens in enumerate(val_iterator) :
            tokens = tokens.to(device)

            _, _, val_loss = model(tokens)
            epoch_val_loss += val_loss.item()

    epoch_train_loss /= len(train_iterator)
    epoch_val_loss /= len(val_iterator)

    return epoch_train_loss, epoch_val_loss

In [ ]:
OPT_PARAMS = copy.copy(DEFAULT_PARAMS)

In [ ]:
# 하이퍼파라미터 최적화를 위해 wandb sweep을 설정 및 실행하는 클래스
class SweepSetting() :
    def __init__(self, sweep_setting, name) :
        wandb_api = wandb.Api()
        sweep_setting['name'] = name

        self.device = DEFAULT_PARAMS['device']
        self.sweep_id = wandb.sweep(sweep=sweep_setting, project=SWEEP_PROJECT)
        self.count = sweep_setting['count']
        self.sweep_output = None
        self.best_run = None

    def copy_params(self, run) :
        params = copy.copy(OPT_PARAMS)
        args = run.config

        def load_config(elem) :
            if args.get(elem) :
                return args[elem]
            else :
                return params[elem]

        return params

    # hyperparameter를 sweep 설정에 따라 변경해가며 최적화된 hyperparameter value를 산출하는 메소드
    def sweep_callback(self) :
        with wandb.init() as run :
            params = self.copy_params(run)
            print(f'params : {params}')

            train_dataloader = torch.utils.data.DataLoader(TRAIN_SET, batch_size=params['batch_size'], shuffle=False, drop_last = True)
            val_dataloader = torch.utils.data.DataLoader(VAL_SET, batch_size=params['batch_size'], shuffle=False, drop_last = True)

            model = TransformerVAE(params).to(self.device)

            optimizer = torch.optim.Adam(model.parameters(), lr=params['learning_rate'])
            epochs = params['epochs']

            for epoch in tqdm(range(1, epochs + 1)) :
                epoch_train_loss, epoch_val_loss = run_train(model, train_dataloader, val_dataloader, optimizer, params['clip'], self.device)
                wandb.log({'epoch': epoch, 'epoch_train_loss': epoch_train_loss, 'epoch_val_loss': epoch_val_loss})

        wandb.finish()
        print('='*150)

    # 스윕 실행
    def start_agent(self) :
        wandb.agent(self.sweep_id, project=SWEEP_PROJECT, function=self.sweep_callback, count=self.count)

    # 스윕 결과로부터 최적화된 hyperparameter 반환
    def get_best_params(self) :
        wandb_api = wandb.Api()
        self.sweep_output = wandb_api.sweep(f'{ENTITY}/{SWEEP_PROJECT}/{self.sweep_id}')
        self.best_run = sorted(self.sweep_output.runs, key=lambda r: r.summary.get('epoch_val_loss', float('inf')))[0]

        temp = json.loads(self.best_run.json_config)
        best_params = copy.copy(OPT_PARAMS)

        for key, value in temp.items() :
                best_params[key] = temp[key]['value']

        return best_params

In [ ]:
# 모델의 학습, 테스트, 파라미터 계산 및 저장, wandb artifact로부터 모델 불러오기를 수행하는 클래스
class Trial() :
    def __init__(self, model, optimizer, params, use_wb, scheduler=None) :
        self.device = OPT_PARAMS['device']
        self.model = model
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.params = params
        self.use_wb = use_wb
        self.clip = OPT_PARAMS['clip']
        self.model_path = '/content/drive/MyDrive/Colab Notebooks/Python/Portfolio/model/'
        self.start_epoch = 1
        self.run = None
        self.report = None

        self.model.apply(self.initialize_weights)

    # Wandb 세션 초기화
    def wandb_init(self) :
        if self.use_wb :
            wandb.finish()

            wandb_api = wandb.Api()
            api_runs = wandb_api.runs(f'{ENTITY}/{TRAIN_PROJECT}')
            run = wandb.init(project=TRAIN_PROJECT, name=f"trial_{len(api_runs)+1}")
            report = wandb_api.run(f'{ENTITY}/{TRAIN_PROJECT}/{run.id}')

            self.run, self.report = run, report

    def initialize_weights(self, m):
        if hasattr(m, 'weight') and m.weight is not None and m.weight.dim() > 1:
            nn.init.xavier_uniform_(m.weight.data)

    # 입력한 run_id에 해당하는 실행 정보를 불러옴
    def wandb_load_run(self, run_id) :
        if self.use_wb :
            wandb_api = wandb.Api()
            run = wandb.init(project=TRAIN_PROJECT, id=run_id, resume=True)
            report = wandb_api.run(f'{ENTITY}/{TRAIN_PROJECT}/{run.id}')

            self.run, self.report = run, report

    # 모델의 학습, 최적화된 모델을 wandb 서버에 저장하는 메소드
    def train(self, train_dataloader, val_dataloader, start_epoch=1) :
        epochs = self.params['epochs']
        period = self.params['period']
        train_loss_list = []
        val_loss_list = []

        today = datetime.now()
        date_str = today.strftime('%y%m%d')
        best_model_name = f'{TRAIN_PROJECT}_best_model_{date_str}.pth'

        # 가장 낮은 validation loss값을 기준으로 모델의 파라미터 저장
        best_val_loss = float('inf')

        for epoch in tqdm(range(start_epoch, epochs + 1)) :
            epoch_train_loss, epoch_val_loss = run_train(self.model, train_dataloader, val_dataloader, self.optimizer, self.clip, self.device )

            train_loss_list.append(epoch_train_loss)
            val_loss_list.append(epoch_val_loss)

            if self.params['use_scheduler'] and self.scheduler != None :
                self.scheduler.step(epoch_val_loss)

            if self.use_wb :
                self.run.log({'epoch': epoch, 'epoch_train_loss': epoch_train_loss, 'epoch_val_loss': epoch_val_loss})

                if epoch_val_loss < best_val_loss :
                    best_val_loss = epoch_val_loss
                    torch.save({
                    'epoch': epoch,
                    'model_state_dict': self.model.state_dict(),
                    'optimizer_state_dict': self.optimizer.state_dict(),
                    'epoch_val_loss': epoch_val_loss,
                    }, f'{self.model_path}{best_model_name}')

            if epoch == 1 or epoch % period == 0 :
                print(f'{"epoch":>12} {epoch:<4} / {epochs:>4}  {"[epoch_train_loss :":>15} {epoch_train_loss:.5f}] {"[epoch_val_loss :":>12} {epoch_val_loss:.5f}] {"[lr :":>8} {self.optimizer.param_groups[0]["lr"]:.8f}]')

        return train_loss_list, val_loss_list

    # 학습된 모델을 테스트하는 메소드
    def test(self, test_dataloader) :

        self.model.eval()
        with torch.no_grad() :
            tokens = next(iter(test_dataloader))
            tokens = tokens.to(self.device)

            _, _, test_loss = self.model(tokens)
            test_loss = test_loss.detach().cpu().numpy()

        if self.use_wb :
            wandb_api = wandb.Api()
            self.run.summary['test_loss'] = test_loss
            self.report = wandb_api.run(f'{ENTITY}/{TRAIN_PROJECT}/{self.run.id}')

        return test_loss

    def count_parameters(self) :
        table = PrettyTable(['Modules', 'Parameters'])
        total_params = 0

        for name, parameter in self.model.named_parameters() :
            if not parameter.requires_grad :
                continue

            param = parameter.numel()
            table.add_row([name, param])
            total_params += param

        print(table)
        print(f'Total Trainable Params : {total_params}')
        return total_params

    def upload(self, best_model_name) :

        if self.use_wb :
            wandb_api = wandb.Api()
            self.report = wandb_api.run(f'{ENTITY}/{TRAIN_PROJECT}/{self.run.id}')

            artifact = wandb.Artifact(best_model_name, type='model')
            artifact.add_file(f'{self.model_path}{best_model_name}')

            self.run.log_artifact(artifact)

    # wandb에 저장한 모델의 파라미터를 불러오는 메소드.
    def load_model(self, file_name, from_wandb=False) :
        if from_wandb :
            model_name = (file_name.split('/')[-1]).split(':')[0]

            artifact = self.run.use_artifact(file_name, type='model')
            artifact_dir = artifact.download()
            artifact_state_dict = torch.load(os.path.join(artifact_dir, model_name))

        else :
            artifact_state_dict = torch.load(f'{self.model_path}{file_name}')

        self.model = TransformerVAE(self.params)
        self.model.load_state_dict(artifact_state_dict['model_state_dict'])
        self.model.to(self.device)

        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.params['learning_rate'])
        self.optimizer.load_state_dict(artifact_state_dict['optimizer_state_dict'])

        self.start_epoch = artifact_state_dict['epoch']

    def gen_mols(self, target_smiles, noise_size) :
        max_len = self.params['selfies_max_len']
        selfies_tokens = self.params['selfies_tokens']
        selfies_reverse_tokens = self.params['selfies_reverse_tokens']
        latent_dim = self.params['latent_dim']

        gen_smiles = torch.zeros((1, max_len))
        gen_smiles[0][0] = selfies_tokens['<sos>']

        for idx, tok in enumerate(selfies.split_selfies(selfies.encoder(target_smiles)), 1) :
            gen_smiles[0][idx] = selfies_tokens[tok]
        gen_smiles[0][idx+1] = selfies_tokens['<eos>']

        gen_src = gen_smiles.type(torch.LongTensor)

        def gen(noise) :
            src_mask = self.model.make_src_mask(gen_src).to(device)
            trg_index = [selfies_tokens['<sos>']]

            for i in range(max_len) :
                trg_tensor = torch.LongTensor(trg_index).unsqueeze(0).to(device)
                trg_mask = self.model.make_trg_mask(trg_tensor).to(device)
                with torch.no_grad() :
                    output, attention = self.model.decoder(trg_tensor, noise, trg_mask, src_mask)
                pred_token = output.argmax(2)[:, -1].item()
                trg_index.append(pred_token)

                if pred_token == selfies_tokens['<eos>'] :
                    break

            result_smiles = ''.join([selfies_reverse_tokens[tok] for tok in trg_index][1:-1])
            return selfies.decoder(result_smiles)


        with torch.no_grad() :
            noise = torch.randn(noise_size, latent_dim).to(device)

        gen_smiles = set()

        for n in range(noise_size) :
            result_smiles = gen(noise[n])

            gen_mol = Chem.MolFromSmiles(result_smiles)
            if gen_mol != None :
                gen_smiles.add(result_smiles)

        return list(gen_smiles)

# Sweep

In [ ]:
dir_path = '/content/drive/MyDrive/Colab Notebooks/jsons/'
login_file_name = 'wandb.json'
login_file_path = os.path.join(dir_path, login_file_name)

with open(login_file_path, 'r') as f :
    json_wandb = json.load(f)

wandb.login(key=json_wandb['key'])
time.sleep(5)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
yaml_files = sorted(os.listdir(os.path.join(dir_path, 'TRANS_VAE_SELFIES')))
list_sweep = []
print(yaml_files)

['1_learning_rate.yaml']


In [ ]:
for idx, yf in enumerate(yaml_files, 1) :
    print(f'\n{"="*70} Condition {idx} {"="*70}\n')
    print('-'*150)
    sweep_setting_name = yf
    sweep_setting_path = os.path.join(dir_path, 'TRANS_VAE_SELFIES', sweep_setting_name)

    with open(sweep_setting_path, 'r') as f :
        sweep_setting = yaml.load(f, Loader=yaml.FullLoader)
        current_sweep = SweepSetting(sweep_setting, sweep_setting_name.split('.')[0])
        current_sweep.start_agent()

        OPT_PARAMS = current_sweep.get_best_params()
        print(f'\nbest hyperparameters : {OPT_PARAMS}\n')
        print('-'*150)

        list_sweep.append(current_sweep)


====================================================================== Condition 1 ======================================================================

------------------------------------------------------------------------------------------------------------------------------------------------------


wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. Additional properties are not allowed ('count' was unexpected)


Create sweep with ID: oqn7k096
Sweep URL: https://wandb.ai/rossi22/TRANS_VAE_SELFIES_SWEEP/sweeps/oqn7k096


wandb: Agent Starting Run: 4f47uz4g with config:
wandb: 	learning_rate: 0.0001
wandb: Currently logged in as: rossi22. Use `wandb login --relogin` to force relogin


params : {'device': device(type='cpu'), 'batch_size': 128, 'learning_rate': 0.0001, 'epochs': 20, 'embed_dim': 32, 'latent_dim': 64, 'hidden_dim': 32, 'enc_layers': 4, 'dec_layers': 4, 'enc_heads': 8, 'dec_heads': 8, 'enc_pos_dim': 256, 'dec_pos_dim': 256, 'enc_dropout': 0.1, 'dec_dropout': 0.1, 'pad_idx': 0, 'clip': 0.01, 'use_scheduler': False, 'selfies_max_len': 34, 'selfies_tokens': {'<pad>': 0, '<sos>': 1, '<eos>': 2, '[Cl]': 3, '[NH1]': 4, '[#C]': 5, '[O]': 6, '[=O]': 7, '[Br]': 8, '[#Branch2]': 9, '[=Ring1]': 10, '[Ring1]': 11, '[S]': 12, '[=Branch2]': 13, '[=Branch1]': 14, '[Ring2]': 15, '[Branch1]': 16, '[C]': 17, '[Branch2]': 18, '[N]': 19, '[=Ring2]': 20, '[P]': 21, '[#Branch1]': 22, '[=S]': 23, '[=C]': 24, '[=N]': 25, '[#N]': 26, '[F]': 27}, 'selfies_reverse_tokens': {0: '<pad>', 1: '<sos>', 2: '<eos>', 3: '[Cl]', 4: '[NH1]', 5: '[#C]', 6: '[O]', 7: '[=O]', 8: '[Br]', 9: '[#Branch2]', 10: '[=Ring1]', 11: '[Ring1]', 12: '[S]', 13: '[=Branch2]', 14: '[=Branch1]', 15: '[Ring2]

...

wandb: Agent Starting Run: kup5jsbt with config:
wandb: 	learning_rate: 0.01


params : {'device': device(type='cpu'), 'batch_size': 128, 'learning_rate': 0.0001, 'epochs': 20, 'embed_dim': 32, 'latent_dim': 64, 'hidden_dim': 32, 'enc_layers': 4, 'dec_layers': 4, 'enc_heads': 8, 'dec_heads': 8, 'enc_pos_dim': 256, 'dec_pos_dim': 256, 'enc_dropout': 0.1, 'dec_dropout': 0.1, 'pad_idx': 0, 'clip': 0.01, 'use_scheduler': False, 'selfies_max_len': 34, 'selfies_tokens': {'<pad>': 0, '<sos>': 1, '<eos>': 2, '[Cl]': 3, '[NH1]': 4, '[#C]': 5, '[O]': 6, '[=O]': 7, '[Br]': 8, '[#Branch2]': 9, '[=Ring1]': 10, '[Ring1]': 11, '[S]': 12, '[=Branch2]': 13, '[=Branch1]': 14, '[Ring2]': 15, '[Branch1]': 16, '[C]': 17, '[Branch2]': 18, '[N]': 19, '[=Ring2]': 20, '[P]': 21, '[#Branch1]': 22, '[=S]': 23, '[=C]': 24, '[=N]': 25, '[#N]': 26, '[F]': 27}, 'selfies_reverse_tokens': {0: '<pad>', 1: '<sos>', 2: '<eos>', 3: '[Cl]', 4: '[NH1]', 5: '[#C]', 6: '[O]', 7: '[=O]', 8: '[Br]', 9: '[#Branch2]', 10: '[=Ring1]', 11: '[Ring1]', 12: '[S]', 13: '[=Branch2]', 14: '[=Branch1]', 15: '[Ring2]

100%|██████████| 20/20 [17:02<00:00, 51.10s/it]


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
epoch_train_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch_val_loss,█▇▅▅▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁
epoch,20
epoch_train_loss,1.38092
epoch_val_loss,1.30576



best hyperparameters : {'device': device(type='cpu'), 'batch_size': 128, 'learning_rate': 0.0001, 'epochs': 20, 'embed_dim': 32, 'latent_dim': 64, 'hidden_dim': 32, 'enc_layers': 4, 'dec_layers': 4, 'enc_heads': 8, 'dec_heads': 8, 'enc_pos_dim': 256, 'dec_pos_dim': 256, 'enc_dropout': 0.1, 'dec_dropout': 0.1, 'pad_idx': 0, 'clip': 0.01, 'use_scheduler': False, 'selfies_max_len': 34, 'selfies_tokens': {'<pad>': 0, '<sos>': 1, '<eos>': 2, '[Cl]': 3, '[NH1]': 4, '[#C]': 5, '[O]': 6, '[=O]': 7, '[Br]': 8, '[#Branch2]': 9, '[=Ring1]': 10, '[Ring1]': 11, '[S]': 12, '[=Branch2]': 13, '[=Branch1]': 14, '[Ring2]': 15, '[Branch1]': 16, '[C]': 17, '[Branch2]': 18, '[N]': 19, '[=Ring2]': 20, '[P]': 21, '[#Branch1]': 22, '[=S]': 23, '[=C]': 24, '[=N]': 25, '[#N]': 26, '[F]': 27}, 'selfies_reverse_tokens': {0: '<pad>', 1: '<sos>', 2: '<eos>', 3: '[Cl]', 4: '[NH1]', 5: '[#C]', 6: '[O]', 7: '[=O]', 8: '[Br]', 9: '[#Branch2]', 10: '[=Ring1]', 11: '[Ring1]', 12: '[S]', 13: '[=Branch2]', 14: '[=Branch1]

#Train&Test

In [ ]:
dir_path = '/content/drive/MyDrive/Colab Notebooks/jsons/'
login_file_name = 'wandb.json'
login_file_path = os.path.join(dir_path, login_file_name)

with open(login_file_path, 'r') as f :
    json_wandb = json.load(f)

wandb.login(key=json_wandb['key'])
time.sleep(5)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
params = copy.copy(OPT_PARAMS)
params['epochs'] = 100
params['period'] = 5
params['schd_patience'] = 10

In [ ]:
train_loader = torch.utils.data.DataLoader(TRAIN_SET, batch_size=params['batch_size'], shuffle=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(VAL_SET, batch_size=params['batch_size'], shuffle=False, drop_last=True)
test_loader = torch.utils.data.DataLoader(TEST_SET, batch_size=len(TEST_SET), shuffle=False, drop_last=False)
print(len(train_loader))

2604


In [ ]:
model = TransformerVAE(params).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=params['learning_rate'])
scheduler = ReduceLROnPlateau(optimizer, factor=params['schd_factor'], patience=params['schd_patience'], min_lr=params['schd_min_lr'])

In [ ]:
trial = Trial(model=model, optimizer=optimizer, params=params, use_wb=True, scheduler=scheduler)
trial.count_parameters()

+----------------------------------------------------------+------------+
|                         Modules                          | Parameters |
+----------------------------------------------------------+------------+
|               encoder.tok_embedding.weight               |    1344    |
|               encoder.pos_embedding.weight               |    2336    |
|              encoder.initial_linear.weight               |    2048    |
|               encoder.initial_linear.bias                |     64     |
|     encoder.en_layers.0.self_attn_layer_norm.weight      |     64     |
|      encoder.en_layers.0.self_attn_layer_norm.bias       |     64     |
|      encoder.en_layers.0.self_attention.fc_q.weight      |    4096    |
|       encoder.en_layers.0.self_attention.fc_q.bias       |     64     |
|      encoder.en_layers.0.self_attention.fc_k.weight      |    4096    |
|       encoder.en_layers.0.self_attention.fc_k.bias       |     64     |
|      encoder.en_layers.0.self_attent

2379562

In [ ]:
trial.wandb_init()
train_loss_list, val_loss_list = trial.train(train_loader, val_loader)

wandb: Currently logged in as: rossi22. Use `wandb login --relogin` to force relogin


  1%|          | 1/100 [03:56<6:29:39, 236.16s/it]

       epoch 1    /  100  [epoch_train_loss : 15.71173] [epoch_val_loss : 1.07550]    [lr : 0.00020000]


  5%|▌         | 5/100 [19:37<6:12:49, 235.47s/it]

       epoch 5    /  100  [epoch_train_loss : 0.93426] [epoch_val_loss : 0.85625]    [lr : 0.00020000]


...

       epoch 90   /  100  [epoch_train_loss : 0.67315] [epoch_val_loss : 0.62936]    [lr : 0.00020000]


 95%|█████████▌| 95/100 [6:11:10<19:21, 232.36s/it]

       epoch 95   /  100  [epoch_train_loss : 0.67081] [epoch_val_loss : 0.62941]    [lr : 0.00020000]


100%|██████████| 100/100 [6:30:33<00:00, 234.33s/it]

       epoch 100  /  100  [epoch_train_loss : 0.66823] [epoch_val_loss : 0.62201]    [lr : 0.00020000]


In [ ]:
trial.upload('TRANS_VAE_SELFIES_TRAIN_best_model_230822.pth')

In [ ]:
test_loss = trial.test(test_loader)
test_loss

array(0.6237212, dtype=float32)

In [ ]:
dutasteride = 'CC12CCC3C(C1CCC2C(=O)NC4=C(C=CC(=C4)C(F)(F)F)C(F)(F)F)CCC5C3(C=CC(=O)N5)C'
duta_selfies = selfies.encoder(dutasteride)
generating_size = 1000

print(f'dutasteride : {dutasteride}')
print(f'dutasteride in dataset? : {duta_selfies in selfies_list}')

gen_mols = trial.gen_mols(dutasteride, generating_size)
print(f'generated mols : {len(gen_mols)}/{generating_size}')

dutasteride : CC12CCC3C(C1CCC2C(=O)NC4=C(C=CC(=C4)C(F)(F)F)C(F)(F)F)CCC5C3(C=CC(=O)N5)C
dutasteride in dataset? : False
generated mols : 265/1000


In [ ]:
print(f'dutasteride : {dutasteride}')
display(Chem.MolFromSmiles(dutasteride))

for i, smiles in enumerate(gen_mols) :
    print(f'\n{i} : {smiles}')
    img = Draw.MolToImage(Chem.MolFromSmiles(smiles))
    display(img)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
wandb.finish()

epoch,100
epoch_train_loss,0.66823
epoch_val_loss,0.62201
test_loss,0.61201


# load trial

In [ ]:
trial.wandb_load_run('t9z44fwe')
trial.load_model('rossi22/TRANS_VAE_SELFIES_TRAIN/TRANS_VAE_SELFIES_TRAIN_best_model_230822.pth:v0', from_wandb=True)
test_loss = trial.test(val_loader)
test_loss

wandb:   1 of 1 files downloaded.  


array(0.6107663, dtype=float32)

In [ ]:
wandb.finish()

epoch,100
epoch_train_loss,0.66823
epoch_val_loss,0.62201
test_loss,0.61077
